# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- script hw_sumatraA_CAN_bp.ipynb
- template for bandpass filtering the vertical seismogram from Sumatra earthquake
- the example waveform is from CAN (Canberra, Australia) for the 2004 Mw 9.1 Sumatra-Andaman earthquake

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings

from lib_seis import smooth
from lib_seis import sumatra_event
from obspy.core import UTCDateTime
from obspy.clients.fdsn import Client 
from scipy.signal import hilbert

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['lines.linewidth'] = 0.5

### Event details
Mw 9.1 2004 Sumatra - Andaman Islands Earthquake \
https://earthquake.usgs.gov/earthquakes/eventpage/official20041226005853450_30

In [ ]:
# fetch and display event details

event = sumatra_event()

print()
for item in event:
    print(f'{item} = {event[item]}')
print()    

In [ ]:
# getting the waveform

client = Client("IRIS")

starttime  = event['origin_time'] - (      4 * 60 )         # 4 minutes before origin time
endtime    = event['origin_time'] + ( 2 * 60 * 60 )         # 2 hours after origin time

st = client.get_waveforms("G", "CAN", "*", "BHZ", starttime, endtime, attach_response=True)
print(st)

In [ ]:
# plotting raw seismogram

can = st.copy()
t   = can[0].times()
tr  = can[0]

title = f'{tr.id} − starting {tr.stats["starttime"]}'

plt.figure()
plt.plot(t, tr.data, c='b')
plt.xlabel('seconds')
plt.ylabel('counts')
plt.title(title)
plt.show()

In [ ]:
# setting up a bandpass filter

T1 = 100   # minimum period: YOU WILL NEED TO CHANGE THIS
T2 = 200   # maximum period: YOU WILL NEED TO CHANGE THIS
f1 = 1/T2
f2 = 1/T1
npoles = 2

can_bp = st.copy()
can_bp.detrend('constant')
can_bp.detrend('linear')
can_bp.taper(0.05)
can_bp.filter('bandpass', freqmin=f1, freqmax=f2, corners=npoles, zerophase=True)

tr1 = can_bp[0]

title = f'{tr1.id} − starting {tr1.stats["starttime"]}'

plt.plot(t, tr1.data, c='b')
plt.xlabel('seconds')
plt.ylabel('counts')
plt.title(title)

In [ ]:
# You will need to apply the functions hilbert AND smooth, FOLLOWING NI ET AL. (2005), FIGURE 1
# hilbert doc: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.hilbert.html

# try something like smooth(tr1,npts)
# where npts is the number of points in a sliding window (MUST BE AN ODD INTEGER)

